In [169]:
!pip install gradio
!pip install google-cloud-storage

In [170]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model # To load the model later to make predictions

In [171]:
# from google.colab import drive
# drive.mount('/content/drive')

In [172]:
# # Try listing the contents of the directory to check if the file exists
# !ls "/content/drive/My Drive/saved_model/"

In [173]:
# import tensorflow as tf

# # Path to your model in GCS
# gcs_model_path = 'gs://gh_gcs_bucket/melanoma_cnn_model.keras'

# # Load the model directly from GCS
# model = tf.keras.models.load_model(gcs_model_path)

# print("Model loaded successfully from GCS!")

In [174]:
# Load the saved model
# file = '/content/drive/My Drive/saved_model/melanoma_cnn_model.keras'
# model = tf.keras.models.load_model(file)


In [175]:
# Convert Image file to numpy file, convert image to floating point & normalize the file and create image_data
def format_image_for_prediction(image):
    # Resize image to 300x300
    image = tf.image.resize(image, (300, 300))

    # Normalize image
    image_normalized = np.array(image).astype(np.float32) / 255
    # Convert values to numpy arrays
    np.array(image_normalized)
    # Add batch dimension
    image_normalized = np.expand_dims(image_normalized, axis=0)  # Shape becomes (1, 300, 300, 3)

    return image_normalized

In [176]:
def get_prediction(image_normalized):
    # # Load the model
    # file = '/content/drive/My Drive/saved_model/melanoma_cnn_model.keras'
    # # model = load_model('saved_model/melanoma_cnn_model.keras')

    try:
        # Path to your model in GCS
        gcs_model_path = 'gs://gh_gcs_bucket/melanoma_cnn_model.keras'

        # Load the model directly from GCS
        model = tf.keras.models.load_model(gcs_model_path)
        # model = load_model(file)
    except Exception as e:
        return ("Error reading model:", str(e))

    # Predict
    try:
      prediction_probs = model.predict(image_normalized)
    except Exception as e:
      return f"Error during prediction: {e}"

    prediction = np.argmax(prediction_probs, axis=1)
    confidence = np.max(prediction_probs)

    # Interpret the prediction
    if prediction == 1:
        return f"Our model's prediction is malignant with {confidence*100:.2f}% confidence."
    else:
        return f"Our model's prediction is benign with {confidence*100:.2f}% confidence."

    # return prediction


In [177]:
def process_image(image):
    # Resize and normalize and convert image to numpy array
    image_normalized = format_image_for_prediction(image)
    # Get prediction
    return get_prediction(image_normalized)

    # try:
    #     # Path to your model in GCS
    #     gcs_model_path = 'gs://gh_gcs_bucket/melanoma_cnn_model.keras'

    #     # Load the model directly from GCS
    #     model = tf.keras.models.load_model(gcs_model_path)
    #     # model = load_model(file)

    #     # return model.compiled_metrics
    #     # return "Model loaded successfully!"
    # except Exception as e:
    #     return "Error reading model:", str(e)

    # target_shape = (300,300,3)  # Original model's input shape
    # if image_normalized.shape[1:] != target_shape:
    #   image_normalized = tf.image.resize(image_normalized, target_shape[:2])

    # try:
    #   prediction_probs = model.predict(image_normalized)
    # except Exception as e:
    #   return f"Error during prediction: {e}"

    # return "Got the prediction"


In [178]:
# Create the Gradio interface with an image upload button
image_scan_app = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil", label="Upload an Image"),
    outputs="text",
    title="Melanoma Image scan app"
  )

image_scan_app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2db799160be039b2d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
